In [1]:
import numpy as np

states = ['E', 'I']  # E: Exon, I: Intron

transition_probs = {
    'E': {'E': 0.9, 'I': 0.1},
    'I': {'E': 0.1, 'I': 0.9}
}

emission_probs = {
    'E': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
    'I': {'A': 0.4, 'C': 0.1, 'G': 0.1, 'T': 0.4}
}

observed_sequence = "CTTCATGTGAAAGCAGACGTAAGTCA"

In [2]:
def get_log_prob_of_a_given_path(path, sequence):
    log_prob = 0
    for i in range(len(sequence)):
        state = path[i]
        nucleotide = sequence[i]
        if i > 0:
            prev_state = path[i - 1]
            log_prob += np.log(transition_probs[prev_state][state])
        log_prob += np.log(emission_probs[state][nucleotide])
    return log_prob



In [3]:
def viterbi_algorithm(sequence):
    n = len(sequence)
    dp = np.zeros((len(states), n))  # Dynamic programming table
    backpointer = np.zeros((len(states), n), dtype=int)  # Backpointer table

    for i, state in enumerate(states):
        dp[i, 0] = np.log(emission_probs[state][sequence[0]])

    for t in range(1, n):
        for i, state in enumerate(states):
            max_prob = float('-inf')
            max_state = 0
            for j, prev_state in enumerate(states):
                prob = dp[j, t - 1] + np.log(transition_probs[prev_state][state]) + np.log(emission_probs[state][sequence[t]])
                if prob > max_prob:
                    max_prob = prob
                    max_state = j
            dp[i, t] = max_prob
            backpointer[i, t] = max_state

    max_final_prob = float('-inf')
    last_state = 0
    for i, state in enumerate(states):
        if dp[i, n - 1] > max_final_prob:
            max_final_prob = dp[i, n - 1]
            last_state = i

    best_path = []
    current_state = last_state
    for t in range(n - 1, -1, -1):
        best_path.append(states[current_state])
        current_state = backpointer[current_state, t]
    best_path.reverse()

    return ''.join(best_path), max_final_prob

path, log_prob = viterbi_algorithm(observed_sequence)
print("Most likely path:", path)
print("Log probability of the path:", log_prob)

Most likely path: EEEEEEEEEEEEEEEEEEEEEEEEEE
Log probability of the path: -38.677666280562796


In [4]:
# Example for get_log_prob_of_a_given_path
example_path = "EEEEEEEEEEEEEEEEEEIIIIIIII"
example_sequence = "CTTCATGTGAAAGCAGACGTAAGTCA"
log_prob_example = get_log_prob_of_a_given_path(example_path, example_sequence)
print("Log probability of the given path:", log_prob_example)

Log probability of the given path: -41.27374490729282
